仓库地址：https://github.com/lijie0610/data_mining/tree/Frequent_patterns-%26-Rules

# 数据集
## 选择Oakland Crime Statistics 2011 to 2016数据集进行分析
## 数据集查看
以2016年为例，查看其属性值和前五列数据。

In [1]:
import numpy as np
import pandas as pd
import os
dataset=pd.read_csv(r'D:\Users\Desktop\data\Oakland Crime Statistics 2011 to 2016\records-for-2016.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110828 entries, 0 to 110827
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Agency                     110827 non-null  object 
 1   Create Time                110827 non-null  object 
 2   Location                   110828 non-null  object 
 3   Area Id                    110827 non-null  object 
 4   Beat                       110247 non-null  object 
 5   Priority                   110827 non-null  float64
 6   Incident Type Id           110827 non-null  object 
 7   Incident Type Description  110827 non-null  object 
 8   Event Number               110827 non-null  object 
 9   Closed Time                110827 non-null  object 
dtypes: float64(1), object(9)
memory usage: 8.5+ MB


In [2]:
dataset.head()

,Agency,Create Time,Location,Area Id,Beat,Priority,Incident Type Id,Incident Type Description,Event Number,Closed Time
0,OP,2016-01-01T00:00:57.000,ST&MARKET ST,P1,05X,2.0,415GS,415 GUNSHOTS,LOP160101000003,2016-01-01T00:32:30.000
1,OP,2016-01-01T00:01:25.000,AV&HAMILTON ST,P3,26Y,2.0,415GS,415 GUNSHOTS,LOP160101000005,2016-01-01T00:48:23.000
2,OP,2016-01-01T00:01:43.000,ST&CHESTNUT ST,P1,02X,2.0,415GS,415 GUNSHOTS,LOP160101000008,2016-01-01T00:21:24.000
3,OP,2016-01-01T00:01:48.000,WALLACE ST,P2,18Y,2.0,415GS,415 GUNSHOTS,LOP160101000007,2016-01-01T01:15:03.000
4,OP,2016-01-01T00:02:05.000,90TH AV,P3,34X,2.0,415GS,415 GUNSHOTS,LOP160101000009,2016-01-01T00:54:52.000


## 数据预处理
将数据转化为关联规则挖掘形式，删除缺失值。
选择Location、Area id、Beat、Incident Type Id、Incident Type Description、Event Number属性进行分析。
缺失值删除后由110828行变为110247行

In [3]:
data=dataset[["Location", "Area Id", "Beat", "Priority", "Incident Type Id", "Event Number"]]
data=data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110247 entries, 0 to 110826
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Location          110247 non-null  object 
 1   Area Id           110247 non-null  object 
 2   Beat              110247 non-null  object 
 3   Priority          110247 non-null  float64
 4   Incident Type Id  110247 non-null  object 
 5   Event Number      110247 non-null  object 
dtypes: float64(1), object(5)
memory usage: 5.9+ MB


# 频繁模式和关联规则挖掘
## 找出频繁模式
## 导出关联规则，计算其支持度和置信度;
## 对规则进行评价，可使用Lift、卡方和其它教材中提及的指标, 至少2种；
挖掘算法为Apriori 算法。
算法步骤如下：
找出出现频率最大的一个项L1。
根据L1找频繁“2项集”的集合C2。
并剪掉不满足支持度阈值的项，得到L2。
根据L2找频繁“3项集”的集合C3。
根据性质和支持度阈值进行剪枝，得到L3。
循环上述过程，直到得到空集C，即直到不能发现更大的频集L。
计算最大频集L的非空子集，两两计算置信度，得到大于置信度阈值的强关联规则。

规则评价选择：Lift和Jaccard

In [4]:
class Apriori():
    def __init__(self,min_support=0.1,min_conf=0.5):
        self.min_support = min_support#支持度阈值
        self.min_conf = min_conf#置信度阈值
    
    def candidate_datasets_l1(self, dataset):       #创建单项频繁项集及L1
        l1 = []
        progress = ProgressBar()
        for data in progress(dataset):
            for item in data:
                if [item] not in l1:
                    l1.append([item])
        return [frozenset(item) for item in l1]
    
    def candidate_datasets_lk(self, dataset):             #根据L1创建多项频繁项集Lk。获取频繁模式。
        l1 = self.candidate_datasets_l1(dataset)        
        dataset = [set(data) for data in dataset]
        frequent_patterns, support = self.lk_low_support_filtering(dataset, l1)
        frequent_patterns = [frequent_patterns]
        k = 2
        while len(frequent_patterns[k-2]) > 0:
            lk = self.apriori_gen(frequent_patterns[k-2], k)        
            Fk, support_k = self.lk_low_support_filtering(dataset, lk)      
            support.update(support_k)
            frequent_patterns.append(Fk)
            k += 1
        return frequent_patterns, support


    def lk_low_support_filtering(self, dataset, lk):   #选择符合支持度的频繁模式
        lk_count = dict()
        for data in dataset:
            for cand in lk:
                if cand.issubset(data):
                    if cand not in lk_count:
                        lk_count[cand] = 1
                    else:
                        lk_count[cand] += 1

        num_items = float(len(dataset))
        fp = []
        support = dict()
        for key in lk_count:
            sup  = lk_count[key] / num_items
            if sup >= self.min_support:
                fp.insert(0, key)
            support[key] = sup
        return fp, support

    def apriori_gen(self, Fk, k):       #合并时检测
        lk = []
        len_Fk = len(Fk)

        for i in range(len_Fk):
            for j in range(i+1, len_Fk):
                F1 = list(Fk[i])[:k-2]
                F2 = list(Fk[j])[:k-2]
                F1.sort()
                F2.sort()
                if F1 == F2:
                    lk.append(Fk[i] | Fk[j])
        return lk

    def change_data_dict(self,datasets):# 将数据转为数据字典
        dataset = []
        col = ["Location", "Area Id", "Beat", "Priority", "Incident Type Id", "Event Number"]
        for data_line in datasets:
            data = []
            for i, value in enumerate(data_line):
                if not value:
                    data.append((col[i], 'NA'))
                else:
                    data.append((col[i], value))
            dataset.append(data)
        return dataset
                
    def get_rules(self, frequent_patterns, support):#获取关联规则，并计算支持度和置信度
        rules = []
        for i in range(1, len(frequent_patterns)):
            for frequent_pattern in frequent_patterns[i]:
                H1 = [frozenset([item]) for item in frequent_pattern]
                if i > 1:
                    self.create_rules(frequent_pattern, H1, support, rules)
                else:
                    self.conviction(frequent_pattern, H1, support, rules)
        return rules

    def create_rules(self, frequent_pattern, H, support, rules):#迭代产生关联规则
        m = len(H[0])
        if len(frequent_pattern) > (m+1):
            Hmp1 = self.apriori_gen(H, m+1)
            Hmp1 = self.conviction(frequent_pattern, Hmp1, support, rules)
            if len(Hmp1) > 1:
                self.create_rules(frequent_pattern, Hmp1, support, rules)

    def conviction(self, frequent_pattern, H, support, rules):          #置信度计算，并对关联规则进行评价
        prunedH = []
        for reasoned_item in H:
            sup = support[frequent_pattern]#支持度
            conf = sup / support[frequent_pattern - reasoned_item]#置信度
            lift = conf / support[reasoned_item]#lift指标
            jaccard = sup / (support[frequent_pattern - reasoned_item] + support[reasoned_item] - sup)#jaccard指标
            if conf >= self.min_conf:
                rules.append((frequent_pattern-reasoned_item, reasoned_item, sup, conf, lift, jaccard))
                prunedH.append(reasoned_item)
        return prunedH
    
    def show_frequent_patterns(self,support):#频繁模式可视化
        for (key, value) in support:
            result_dict = {'frequent_pattern': None, 'support': None}
            set_result = list(key)
            sup_result = value
            if sup_result < self.min_support:
                continue
            result_dict['frequent_pattern'] = set_result
            result_dict['support'] = sup_result
            print(result_dict)
            print('\n')
    
    def show_rules(self,rules):#关联规则可视化
        for result in rules:
            result_dict = {'reason': None, 'result': None, 'support': None, 'conviction': None, 'lift': None, 'jaccard': None}
            X_set, Y_set, support, conf, lift, jaccard = result
            result_dict['reason'] = list(X_set)
            result_dict['result'] = list(Y_set)
            result_dict['support'] = support
            result_dict['conviction'] = conf
            result_dict['lift'] = lift
            result_dict['jaccard'] = jaccard
            print(result_dict)
            print('\n')

In [5]:
from progressbar import *
apriori = Apriori()
datasets = data.values.tolist()
dataset=apriori.change_data_dict(datasets)
# 找出频繁模式
frequent_patterns, support = apriori.candidate_datasets_lk(dataset)
print("support ", support)
# 导出关联规则，及其支持度和置信度
rules = apriori.get_rules(frequent_patterns, support)
rules = sorted(rules, key=lambda x: x[3], reverse=True)
print("rules ", rules)

100% |########################################################################|
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 结果可视化展示

In [6]:
#频繁模式可视化
support = sorted(support.items(), key=lambda d: d[1], reverse=True)
apriori.show_frequent_patterns(support)

{'frequent_pattern': [('Priority', 2.0)], 'support': 0.7781073407893185}


{'frequent_pattern': [('Area Id', 'P3')], 'support': 0.4289731239852332}


{'frequent_pattern': [('Area Id', 'P1')], 'support': 0.37308951717507055}


{'frequent_pattern': [('Area Id', 'P3'), ('Priority', 2.0)], 'support': 0.32334666702948833}


{'frequent_pattern': [('Area Id', 'P1'), ('Priority', 2.0)], 'support': 0.2964615817210446}


{'frequent_pattern': [('Priority', 1.0)], 'support': 0.22189265921068146}


{'frequent_pattern': [('Area Id', 'P2')], 'support': 0.17775540377515942}


{'frequent_pattern': [('Area Id', 'P2'), ('Priority', 2.0)], 'support': 0.13877021596959555}


{'frequent_pattern': [('Area Id', 'P3'), ('Priority', 1.0)], 'support': 0.10562645695574482}




In [7]:
#关联规则可视化
apriori.show_rules(rules)

{'reason': [('Area Id', 'P1')], 'result': [('Priority', 2.0)], 'support': 0.2964615817210446, 'conviction': 0.7946124671788388, 'lift': 1.0212118888028705, 'jaccard': 0.34684608201035744}


{'reason': [('Area Id', 'P2')], 'result': [('Priority', 2.0)], 'support': 0.13877021596959555, 'conviction': 0.7806807164361892, 'lift': 1.003307224481728, 'jaccard': 0.1698341511067694}


{'reason': [('Area Id', 'P3')], 'result': [('Priority', 2.0)], 'support': 0.32334666702948833, 'conviction': 0.7537690567314401, 'lift': 0.9687211740822423, 'jaccard': 0.36588695357645057}




# 结果分析

根据结果可知，我们可以挖掘地区和犯罪数量和案件严重性的关系。
通过观察频繁模型可以看出，Area Id 为 p3 的支持度较高，该地区的犯罪案件数量最多。
通过观察关联规则得知，Area Id：P1与Priority：2.0的置信度较高，说明该地区的案件较为严重。